In [1]:
import torch as th
from torch import optim, nn

In [33]:
# toy dataset
data = th.tensor([[1., 1], [0, 1], [1, 0], [0, 0]], requires_grad=True)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True)

In [34]:
#Simple linear model
model = nn.Linear(2, 1)

In [35]:
#create optimizer

opt = optim.SGD(params=model.parameters(), lr = 0.01)

In [36]:
opt.zero_grad()

In [37]:
pred = model(data)

In [38]:
loss = ((pred - target) ** 2).sum()

In [39]:
loss.backward()

In [40]:
opt.step()

In [41]:
loss.data

tensor(4.8921)

In [42]:
#create method to learn the model

def train(iteration = 20):
    for i in range(iteration):
        opt.zero_grad()
        pred = model(data)
        loss = ((pred - target) ** 2).sum()
        loss.backward()
        opt.step()
        print("In %i loss:"%i, "%f" %loss.data)
        

In [43]:
train()

In 0 loss: 3.795036
In 1 loss: 2.957207
In 2 loss: 2.316843
In 3 loss: 1.826911
In 4 loss: 1.451601
In 5 loss: 1.163645
In 6 loss: 0.942279
In 7 loss: 0.771690
In 8 loss: 0.639838
In 9 loss: 0.537552
In 10 loss: 0.457847
In 11 loss: 0.395402
In 12 loss: 0.346165
In 13 loss: 0.307045
In 14 loss: 0.275690
In 15 loss: 0.250305
In 16 loss: 0.229521
In 17 loss: 0.212295
In 18 loss: 0.197829
In 19 loss: 0.185514


In [45]:
# Use two virtual workers and send half datasets to each
import syft as sy

hook = sy.TorchHook(th)
viper = sy.VirtualWorker(hook, id = "viper")
quassi = sy.VirtualWorker(hook, id = "quassi")

In [46]:
data_viper = data[:2].send(viper)
target_viper = target[:2].send(viper)

data_quassi = data[2:].send(quassi)
target_quassi = target[2:].send(quassi)

In [47]:
datasets = [(data_viper, target_viper), (data_quassi, target_quassi)]

In [48]:
model = nn.Linear(2, 1)

In [49]:
opt = optim.SGD(params = model.parameters(), lr = 0.01)

In [50]:
_data, _target = datasets[0]
model = model.send(_data.location)

In [51]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [53]:
train()

tensor(7.7274, requires_grad=True)
tensor(0.2678, requires_grad=True)
tensor(0.2860, requires_grad=True)
tensor(0.2533, requires_grad=True)
tensor(0.1066, requires_grad=True)
tensor(0.1542, requires_grad=True)
tensor(0.0598, requires_grad=True)
tensor(0.0917, requires_grad=True)
tensor(0.0347, requires_grad=True)
tensor(0.0546, requires_grad=True)
tensor(0.0203, requires_grad=True)
tensor(0.0327, requires_grad=True)
tensor(0.0119, requires_grad=True)
tensor(0.0197, requires_grad=True)
tensor(0.0070, requires_grad=True)
tensor(0.0119, requires_grad=True)
tensor(0.0041, requires_grad=True)
tensor(0.0073, requires_grad=True)
tensor(0.0025, requires_grad=True)
tensor(0.0045, requires_grad=True)
tensor(0.0015, requires_grad=True)
tensor(0.0028, requires_grad=True)
tensor(0.0009, requires_grad=True)
tensor(0.0017, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(0.0011, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0002, requi